In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datasets\imputed_data_QS.csv', encoding='latin1')
df['Institution Name'] = df['Institution Name'].str.lower()
df.head()

,Rank,Institution Name,Country,Year,CS,CF,RI,Age,STA,AR_SCORE,ER_SCORE,FS_SCORE,CPF_SCORE,IF_SCORE,IS_SCORE,IRN_SCORE,EO_SCORE,S_SCORE,Overall
0,1.0,massachusetts institute of technology (mit),United States,2018,2.0,2.0,4.0,5.0,2.0,100.0,100.0,100.0,99.9,100.0,96.1,85.943549,98.410782,93.046961,100.0
1,2.0,stanford university,United States,2018,3.0,3.0,4.0,5.0,2.0,100.0,100.0,100.0,99.4,99.6,72.7,96.078472,99.199191,94.038122,98.7
2,3.0,harvard university,United States,2018,3.0,3.0,4.0,5.0,2.0,100.0,100.0,98.3,99.9,96.5,75.2,95.953509,99.380323,93.665746,98.4
3,4.0,california institute of technology (caltech),United States,2018,1.0,2.0,4.0,5.0,2.0,99.5,85.4,100.0,100.0,93.4,89.2,67.229715,69.589757,73.524862,97.7
4,5.0,university of cambridge,United Kingdom,2018,3.0,3.0,4.0,5.0,3.0,100.0,100.0,100.0,78.3,97.4,97.7,97.110243,99.048787,86.625967,95.6


### Create Rank for Each Features

In [3]:
def rank_summary(df):
    rank_columns = ['AR_SCORE', 'ER_SCORE', 'FS_SCORE', 'CPF_SCORE','IF_SCORE','IS_SCORE','IRN_SCORE','EO_SCORE','S_SCORE','Overall']

    for col in rank_columns:
        df[f'{col}_RANK'] = df[col].rank(method='min', ascending=False).astype(int)
    return df

In [4]:
df_rank = rank_summary(df)
df_rank.head()

,Rank,Institution Name,Country,Year,CS,CF,RI,Age,STA,AR_SCORE,...,AR_SCORE_RANK,ER_SCORE_RANK,FS_SCORE_RANK,CPF_SCORE_RANK,IF_SCORE_RANK,IS_SCORE_RANK,IRN_SCORE_RANK,EO_SCORE_RANK,S_SCORE_RANK,Overall_RANK
0,1.0,massachusetts institute of technology (mit),United States,2018,2.0,2.0,4.0,5.0,2.0,100.0,...,1,1,1,62,1,601,1010,94,213,1
1,2.0,stanford university,United States,2018,3.0,3.0,4.0,5.0,2.0,100.0,...,1,1,1,120,596,1490,166,63,177,16
2,3.0,harvard university,United States,2018,3.0,3.0,4.0,5.0,2.0,100.0,...,1,1,318,62,980,1404,177,59,191,23
3,4.0,california institute of technology (caltech),United States,2018,1.0,2.0,4.0,5.0,2.0,99.5,...,119,511,1,1,1189,920,2835,698,1315,31
4,5.0,university of cambridge,United Kingdom,2018,3.0,3.0,4.0,5.0,3.0,100.0,...,1,1,1,814,901,481,112,70,491,52


### Variation Analysis of Ranking

In [5]:
def compute_growth_for_group(df, group_column, year_column, value_column, output_column):
    df[output_column] = None  # 创建空列
    for name, group in df.groupby(group_column):
        group = group.sort_values(year_column)  # 保留原始索引
        for i in range(1, len(group)):
            prev_year = group.iloc[i - 1][year_column]
            curr_year = group.iloc[i][year_column]
            if curr_year - prev_year not in [1, 2]:
                continue
            prev_value = group.iloc[i - 1][value_column]
            curr_value = group.iloc[i][value_column]
            if pd.notnull(prev_value) and prev_value != 0:
                growth = (curr_value - prev_value) / prev_value
            else:
                growth = None
            # 直接使用原始索引
            df_index = group.index[i]
            df.loc[df_index, output_column] = growth
    df[output_column] = df[output_column].astype(float) * 100
    return df

In [6]:
def compute_growth_for_group_2year(df, group_column, year_column, value_column, output_column):
    df[output_column] = None  # 初始化输出列
    for name, group in df.groupby(group_column):
        group = group.sort_values(year_column)  # 保留原始索引
        for i in range(1, len(group)):
            prev_year = group.iloc[i - 1][year_column]
            curr_year = group.iloc[i][year_column]
            # 仅接受恰好3年间隔的对比（如2022 vs 2019）
            if curr_year - prev_year != 2:
                continue
            prev_value = group.iloc[i - 1][value_column]
            curr_value = group.iloc[i][value_column]
            if pd.notnull(prev_value) and prev_value != 0:
                growth = (curr_value - prev_value) / prev_value
            else:
                growth = None
            df_index = group.index[i]
            df.loc[df_index, output_column] = growth
    df[output_column] = pd.to_numeric(df[output_column], errors='coerce') * 100  # 转为百分比
    return df

In [7]:
columns_to_process = {
    'AR_SCORE': 'AR_SCORE_Gro1',
    'AR_SCORE_RANK': 'AR_SCORE_RANK_Gro1',
    'ER_SCORE': 'ER_SCORE_Gro1',
    'ER_SCORE_RANK': 'ER_SCORE_RANK_Gro1',
    'FS_SCORE': 'FS_SCORE_Gro1',
    'FS_SCORE_RANK': 'FS_SCORE_RANK_Gro1',
    'CPF_SCORE': 'CPF_SCORE_Gro1',
    'CPF_SCORE_RANK': 'CPF_SCORE_RANK_Gro1',
    'IF_SCORE': 'IF_SCORE_Gro1',
    'IF_SCORE_RANK': 'IF_SCORE_RANK_Gro1',
    'IS_SCORE': 'IS_SCORE_Gro1',
    'IS_SCORE_RANK': 'IS_SCORE_RANK_Gro1',
    'IRN_SCORE': 'IRN_SCORE_Gro1',
    'IRN_SCORE_RANK': 'IRN_SCORE_RANK_Gro1',
    'EO_SCORE': 'EO_SCORE_Gro1',
    'EO_SCORE_RANK': 'EO_SCORE_RANK_Gro1',
    'S_SCORE': 'S_SCORE_Gro1',
    'S_SCORE_RANK': 'S_SCORE_RANK_Gro1',
    'Overall': 'Overall_Gro1',
    'Overall_RANK': 'Overall_RANK_Gro1'
}

for value_col, output_col in columns_to_process.items():
    df1_updated = compute_growth_for_group(
        df_rank,
        group_column='Institution Name',
        year_column='Year',
        value_column=value_col,
        output_column=output_col
    )
columns_to_process2 = {
    'AR_SCORE': 'AR_SCORE_Gro2',
    'AR_SCORE_RANK': 'AR_SCORE_RANK_Gro2',
    'ER_SCORE': 'ER_SCORE_Gro2',
    'ER_SCORE_RANK': 'ER_SCORE_RANK_Gro2',
    'FS_SCORE': 'FS_SCORE_Gro2',
    'FS_SCORE_RANK': 'FS_SCORE_RANK_Gro2',
    'CPF_SCORE': 'CPF_SCORE_Gro2',
    'CPF_SCORE_RANK': 'CPF_SCORE_RANK_Gro2',
    'IF_SCORE': 'IF_SCORE_Gro2',
    'IF_SCORE_RANK': 'IF_SCORE_RANK_Gro2',
    'IS_SCORE': 'IS_SCORE_Gro2',
    'IS_SCORE_RANK': 'IS_SCORE_RANK_Gro2',
    'IRN_SCORE': 'IRN_SCORE_Gro2',
    'IRN_SCORE_RANK': 'IRN_SCORE_RANK_Gro2',
    'EO_SCORE': 'EO_SCORE_Gro2',
    'EO_SCORE_RANK': 'EO_SCORE_RANK_Gro2',
    'S_SCORE': 'S_SCORE_Gro2',
    'S_SCORE_RANK': 'S_SCORE_RANK_Gro2',
    'Overall': 'Overall_Gro2',
    'Overall_RANK': 'Overall_RANK_Gro2'
}


for value_col, output_col in columns_to_process2.items():
    df1_updated = compute_growth_for_group_2year(
        df_rank,
        group_column='Institution Name',
        year_column='Year',
        value_column=value_col,
        output_column=output_col
    )

df1_updated.to_csv('datasets\Rank_Variation.csv', index=False)

In [8]:
df1 = pd.read_csv('datasets\Rank_Variation.csv', encoding='latin1')
df1 = df1.loc[df1['Year']>=2022]
df1.iloc[:, [1,3] + list(range(29, 69))]

,Institution Name,Year,AR_SCORE_Gro1,AR_SCORE_RANK_Gro1,ER_SCORE_Gro1,ER_SCORE_RANK_Gro1,FS_SCORE_Gro1,FS_SCORE_RANK_Gro1,CPF_SCORE_Gro1,CPF_SCORE_RANK_Gro1,...,IS_SCORE_Gro2,IS_SCORE_RANK_Gro2,IRN_SCORE_Gro2,IRN_SCORE_RANK_Gro2,EO_SCORE_Gro2,EO_SCORE_RANK_Gro2,S_SCORE_Gro2,S_SCORE_RANK_Gro2,Overall_Gro2,Overall_RANK_Gro2
2851,massachusetts institute of technology (mit),2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2852,university of oxford,2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.341204,-54.692557,...,0.000000,0.000000,-0.205421,12.676056,-0.058659,1.265823,9.377300,-64.126394,2.156057,-70.588235
2853,stanford university,2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.318458,-64.971751,...,-1.033973,1.315789,0.063998,-4.591837,-0.149011,4.918033,-0.139202,4.736842,-1.300000,1500.000000
2854,university of cambridge,2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.123989,-55.711207,...,0.102459,-1.635992,0.923323,-38.095238,1.603230,-56.603774,7.970968,-59.557344,2.919708,-67.346939
2855,harvard university,2022,0.000000,0.000000,0.000000,0.000000,0.405268,-9.364548,0.401606,-99.065421,...,12.700965,-14.585576,0.809877,-26.605505,0.194073,-40.425532,-1.816364,20.618557,-0.406504,21.739130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9548,university of west bohemia,2026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9549,voronezh state university,2026,5.555556,-1.627184,74.074074,-10.445070,43.577982,-22.175168,23.076923,-3.609088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9550,walailak university,2026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9551,yamaguchi university,2026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
